In [ ]:
%load_ext autoreload
%autoreload 2

# Sonecule: TVOscBankPMS – Time-variant Oscillator Bank Parameter Mapping Sonification

This notebook introduces and demonstrates usage of the TVOscBankPMS sonecule.

The sonecule uses a bank of continous sine oscillators to represent the channels of multivariate data,
- mapping each value to a pitch deviation from the oscillators base (central) frequency
  - ampmode=="val"/"absval": mapping each value or absolute value to amplitude
  - ampmode=="change": mapping the absolute change to amplitude
- parameters:
  - base_pitch: midi note number of the first channel
  - delta_pitch: 
    - if int: channel separation in semitones
    - if array: offsets in...  

In [ ]:
# headers and imports for the demo
import sonecules as sn
import sc3nb as scn
from pya import Asig
import pyamapping as pam
import matplotlib.pyplot as plt

# setup for matplotlib 
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

# start sonecules (with default backend sc3nb, aka sc3)
sn.startup()
ctx = sn.gcc()  # get the context as ctx

In [ ]:
# sn.gcc().backend.sc.exit()

Load data sets used for the demo

In [ ]:
%run ../examples/prepare-data.ipynb

In [ ]:
# select test data for sonecule, here EEG data from an epilepsy
dasig = Asig(eeg_data, sr=250)
plt.figure(figsize=(12,2)); 
plt.subplot(121); dasig.plot(offset=1)

data = dasig[{7:11}, [0,1,2,5,9,12]][::5]
plt.subplot(122); data.plot(offset=2)

## Usage Demo for the TVOscBankPMS Sonecule

In [ ]:
from mesonic.synth import Synth
from sonecules.scoresyn import TVOscBankPMS

The following code cell shows everything needed 
- to create the sonecule with data, 
- to clear the auditory canvas (aka timeline)
- to start the playback at a given rate
- to plot the timeline.

In [ ]:
# create a sonecule, initialized with your data selection
snctvo = TVOscBankPMS(dasig[{7.5: 11.5}, [1, 2, 3]][::2])

# clear the timeline (if needed, initially it is empty anyway)
ctx.timeline.reset() 

# as needed schedule specific sonifications using that data, specifying method parameters as needed
snctvo.schedule(at=0, rate=2, base_pitch=60, pitch_step=12, pitch_relwid=0.1,
    amp_mode="change", level=-20, map_mode="channelwise")

# start the realtime playback at a given rate
snctvo.start(rate=1)

# plot the timeline
ctx.timeline.plot()

Note that the events remain in the timeline. 
Setting the 

In [ ]:
ctx.timeline

Setting the time actively to 0 will cue the playback to that onset and result in
a sonification to be replayed

In [ ]:
ctx.playback.time = 0

In [ ]:
# to free the timeline use
ctx.clear()

In [ ]:
# to stop all sound playing via the backend use 
ctx.stop()

In [ ]:
# note that the playback's latency is >0 - it can also be set
# but see mesonic for details and help
ctx.playback.processor.latency

In [ ]:
# we can play now interactaively sonify the data with different parameters.
ctx.timeline.reset()

snctvo.schedule(
    at=0, rate=0.5, base_pitch=50, pitch_step=24, pitch_relwid=0.5, 
    amp_mode="change", level=-10, map_mode="channelwise",
).start(rate=1)

the above code demonstrates how the values can also modulate pitch deviations,
- here specifically by 0.5 = 50% of the inter-base pitch deviations

In [ ]:
ctx.timeline.reset()

snctvo.schedule(
    at=0, rate=1, base_pitch=80, pitch_step=2, pitch_relwid=0, 
    amp_mode="change", level=-20, map_mode="channelwise"
).start()

* Instead of specifying a constant pitch step between channel tones we can specifiy an array of pitches to be used.
* This can result in a 'musical/harmonic' sonification, e.g. by setting the individual channels to the notes of a major chord.
* This technique has been called Polyphonic Time Series Sonification and was used for ECG data.
* Now it can be flexibly recreated in few lines of code 
* The following example illustrates such a special case

In [ ]:
snctvo = TVOscBankPMS(dasig[{35:40}, :6][::5])
ctx.timeline.reset()
snctvo.schedule(
    at=0, rate=1, base_pitch=50, pitch_step=[0,4,7,12,16,19], pitch_relwid=0, 
    amp_mode="change", level=-10, map_mode="channelwise"
).start()

* In TVOsc, the spectral position is specified by pitch and pitch step
* What if we want use use specific frequencies, e.g. harmonics of a fundamental?
* with the current interface, some computations are required as shown below
* later versions may offer other keywords such as base_freq and freq_step,
* or offer to set base_pitch to None in which case a kwarg freq_step as an array of frequenies would be expected. 
* Such extensions are planned for future versions. Suggestions for a flexible but clear API are welcome!

In [ ]:
# timbral sonification is just a special case of TVOSon
ctx.timeline.reset()
snctvo = TVOscBankPMS(dasig[{5: 45}, :][::5])
f0 = 40
base_pitch = pam.cps_to_midi(f0)
pitch_steps = [ pam.cps_to_midi(f0*(i+1)) - base_pitch for i in range(dasig.channels)] 
snctvo.schedule(at=0, rate=5, base_pitch=base_pitch, pitch_step=pitch_steps, pitch_relwid=0,  # use 1.5 for pitch added effect
    amp_mode="change", level=-10, map_mode="channelwise",
).start(rate=1)

Auditory Graph as a function sonification is a special case of TVOscBankPMS as well

Let's create data points of a math function to be represented as auditory graph.
* let's put it into an Asig an plot it


In [ ]:
# here create a mathematical function as data
def fn(x):
    return x**2 - 4*x + 1*np.sin(15*x**0.8)
data = fn(np.linspace(0, 5, 100))
dasig = Asig(data, sr=100)
plt.figure(figsize=(8,2)); dasig.plot(offset=1)

In [ ]:
# now sonify it as Auditory Graph using the TVOscBankPMS sonecule
snctvo = TVOscBankPMS(dasig)  
ctx.timeline.reset() 
snctvo.schedule(at=0, rate=0.2, base_pitch=72, pitch_step=30, pitch_relwid=1,
    amp_mode="value", level=-20, map_mode="channelwise")
snctvo.start(rate=1)

## Code Template

The following code snippets are intended for copy & paste to your notebooks, to facilitate getting your data sonified
using this sonecule.
* It is assumed that your data is stored in an Asig dasig

In [ ]:
# load your multi-channel data into an Asig, e.g. 
data = np.random.random((100, 12)) # 100 rows with 8 channels, here same fake data
dasig = Asig(data, sr=10)
plt.figure(figsize=(8,2)); dasig.plot(offset=1)

In [ ]:
# create a sonecule, initialized with your data selection
snctvo = TVOscBankPMS(dasig[{None:None}, :])  # using timeslice to access subset

# clear the timeline (if needed, initially it is empty anyway)
ctx.timeline.reset() 

# as needed schedule specific sonifications using that data, specifying method parameters as needed
snctvo.schedule(at=0, rate=4, base_pitch=40, pitch_step=5, pitch_relwid=1.5,
    amp_mode="change", level=-20, map_mode="channelwise")

# start the realtime playback at a given rate
snctvo.start(rate=1)

# if needed: plot the timeline using `ctx.timeline.plot()`